# Cutting plane and bundle methods
The second part of the tutorial focuses on cutting plane and bundle
methods. We aim at resolving the following LASSO problem:
$$
\min_r \;f(r) =  \| A u - b \|_2^2 + \lambda \|u \|_1
$$
with $\lambda$ a given regularization parameter, $X$ and $b$ input data.

## Settings
We import the usual packages:

In [1]:
using Printf, Random
using LinearAlgebra
using ForwardDiff
using JuMP, CPLEX, OSQP

Fix seed

In [2]:
Random.seed!(2713);

Some constants

In [3]:
const LB = -1e20
const UB =  1e20
const SOLVER = OSQP.Optimizer

OSQP.MathOptInterfaceOSQP.Optimizer

We first generate artificial data to study the algorithm:

In [4]:
include("data.jl")

nVariables = 10;
nCassures = 10;
xMin, xMax = build_bounds(nVariables)
A = build_X(nVariables, nCassures, xMin, xMax);
b = randn(nCassures);
λ = 50.0;

Build oracle for objective:

In [5]:
f(u) = 0.5 * norm(A*u - b, 2)^2 + λ * norm(u, 1);

Function that gets the actual solution:

In [6]:
function get_solution(A, b, λ)
    n, m = size(A)
    model = Model(with_optimizer(SOLVER))
    JuMP.set_silent(model)
    # Variable u
    @variable(model, u[1:n])
    # Variable t, linearizing the \|.\|_1 term
    @variable(model, t[1:n])
    @constraint(model, t .>= u)
    @constraint(model, t .>= -u)
    @objective(model, Min, 0.5 * dot(A*u - b, A*u - b) + λ * sum(t))
    JuMP.optimize!(model)
    return JuMP.objective_value(model)
end
optsol = get_solution(A, b, λ)
println("Optimal solution is equal to ", optsol)

Optimal solution is equal to 5.054101320280825


## A note on the choice of solvers : 

Despite CPLEX being the faster solver of the two, we generally preferred using OSQP here due to numerical issues. In particular, here are a couple of our observations.
- CPLEX (sometimes ?) does not deal very well with the very low lower bound LB. It appears that the bound is so low that it believes the problem to be unbounded (especially using the bundle method). Here we have an obvious reasonable lower bound of 0, but that might not be the case in other applications.
- In the bundle method, even when setting $α \geq 0$ instead of LB, we have numerical instability depending on the precision threshold "tol". If too precise, CPLEX will also run into a problem it deems impossible to solve, and the algorithm will terminate too early.


## Cutting plane
The cutting plane method builds a proxy $\underline{f}_k$ for the original
function $f$, such that $\underline{f}_k$ is polyhedral and is a lower approximation:
$$
\underline{f}_k \leq f \quad \forall k
$$
If we have at disposal a collection of point $x_1, \cdots, x_k$,
with associated subgradients $g_1, \cdots, g_k$, the function
$\underline{f}_k$ writes out
$$
\begin{aligned}
f_k(x) = min_x \;& \theta  \\
         s.t. \quad & \theta \geq g_k^\top (x - x_k) + f(x_k)
\end{aligned}
$$

We can define the cutting plane algorithm as follows.

In [38]:
function launch_cutting_plane(X, xMin, xMax, maxit=5000)
    master = Model(with_optimizer(SOLVER))
    JuMP.set_silent(master)
    #set_parameter(master, "CPX_PARAM_LPMETHOD", 2) #For CPLEX
    
    @variable(master, α >= LB)
    #@variable(master, α >= 0) #For CPLEX
    @variable(master, xMin[i] <= x[i in 1:size(X,1)] <= xMax[i])
    @objective(master, Min, α)
    
    lb, ub = LB, UB

    best_ub = ub

    EPS = 1e-6
    
    trace_f = Float64[]
    trace_ub = Float64[]
    trace_α = Float64[]
    
    for n_iter in 1:maxit
        println("Iteration : ",n_iter)
        JuMP.optimize!(master)
        lb = JuMP.value(α)
        x_k = JuMP.value.(x)
        f_k = f(x_k)
        println("Lower bound : ", lb)
        println("Valeur au point courant : ", f_k)
        g_k = ForwardDiff.gradient(f, x_k)
        #push!(trace_f, f_k)
        #push!(trace_ub, best_ub)
        #push!(trace_α, lb)
        if (f_k - lb <= EPS)
            return lb
            break
        else
            @constraint(master, α >= f_k + sum(g_k[i]*(x-x_k)[i] for i in 1:size(X,1)))
        end
    end
    return lb
end;

In [36]:
launch_cutting_plane(A,xMin,xMax)

Iteration : 1
Lower bound : -3.1800003999959984e9
Valeur au point courant : 5.5995940672380655
Iteration : 2
Lower bound : -1.4457933214513178e6
Valeur au point courant : 3.3675226851362384e11
Iteration : 3
Lower bound : -6036.382987801023
Valeur au point courant : 4.830017132929059e6
Iteration : 4
Lower bound : -5324.702056530599
Valeur au point courant : 2.213038209601912e6
Iteration : 5
Lower bound : -4405.6518922191135
Valeur au point courant : 1.802289882269235e6
Iteration : 6
Lower bound : -4143.409360212969
Valeur au point courant : 1.5539250988131294e6
Iteration : 7
Lower bound : -3844.3852966462514
Valeur au point courant : 1.0845863910559106e6
Iteration : 8
Lower bound : -3722.1760063438564
Valeur au point courant : 766315.4607480648
Iteration : 9
Lower bound : -3150.2630384047566
Valeur au point courant : 976419.2499066555
Iteration : 10
Lower bound : -3044.784385249375
Valeur au point courant : 688163.3264318173
Iteration : 11
Lower bound : -2951.268357340431
Valeur au poin

Lower bound : -231.79320604561892
Valeur au point courant : 3825.350821801569
Iteration : 93
Lower bound : -216.86926259740096
Valeur au point courant : 3699.1413941214596
Iteration : 94
Lower bound : -210.42703498220106
Valeur au point courant : 2201.3236024697567
Iteration : 95
Lower bound : -207.74263140560043
Valeur au point courant : 4505.089306680495
Iteration : 96
Lower bound : -198.42739997335195
Valeur au point courant : 5191.108877783507
Iteration : 97
Lower bound : -197.7193585909616
Valeur au point courant : 2220.779935961791
Iteration : 98
Lower bound : -180.38536296123652
Valeur au point courant : 4897.82193433225
Iteration : 99
Lower bound : -173.8284890578014
Valeur au point courant : 2205.805836583342
Iteration : 100
Lower bound : -165.22384274323028
Valeur au point courant : 3211.8682922031317
Iteration : 101
Lower bound : -161.79618951092561
Valeur au point courant : 3203.770902771105
Iteration : 102
Lower bound : -160.38729951719603
Valeur au point courant : 1407.78

Lower bound : -3.678841090014681
Valeur au point courant : 24.661738879132088
Iteration : 181
Lower bound : -2.738935529044464
Valeur au point courant : 21.472818765966714
Iteration : 182
Lower bound : -2.2499384390653714
Valeur au point courant : 27.073122256745116
Iteration : 183
Lower bound : -2.03302935221937
Valeur au point courant : 23.78661305253918
Iteration : 184
Lower bound : -1.2227863123159755
Valeur au point courant : 18.085663095821282
Iteration : 185
Lower bound : -1.3461695990988551
Valeur au point courant : 18.120658695179852
Iteration : 186
Lower bound : -0.24949118268232964
Valeur au point courant : 12.559017894169632
Iteration : 187
Lower bound : 0.18074398414938703
Valeur au point courant : 10.004051385368015
Iteration : 188
Lower bound : 0.2616125971426604
Valeur au point courant : 16.491965225590732
Iteration : 189
Lower bound : 0.6384280188239808
Valeur au point courant : 14.293312126238028
Iteration : 190
Lower bound : 0.9299419260632986
Valeur au point courant

Lower bound : 5.053671508251579
Valeur au point courant : 5.0579484765367075
Iteration : 272
Lower bound : 5.05360741036451
Valeur au point courant : 5.057941134625597
Iteration : 273
Lower bound : 5.053272421099027
Valeur au point courant : 5.05652760022363
Iteration : 274
Lower bound : 5.053098176798208
Valeur au point courant : 5.057208896827271
Iteration : 275
Lower bound : 5.053330546412207
Valeur au point courant : 5.056635477985908
Iteration : 276
Lower bound : 5.051626492294702
Valeur au point courant : 5.05928724985713
Iteration : 277
Lower bound : 5.051806296661028
Valeur au point courant : 5.057736822030141
Iteration : 278
Lower bound : 5.052074854507474
Valeur au point courant : 5.057337551017559
Iteration : 279
Lower bound : 5.0528279306523
Valeur au point courant : 5.056186941885195
Iteration : 280
Lower bound : 5.0526828383842615
Valeur au point courant : 5.062757968854112
Iteration : 281
Lower bound : 5.0533713282321475
Valeur au point courant : 5.055859320658802
Iterat

Iteration : 363
Lower bound : 5.053515480449823
Valeur au point courant : 5.054550601082633
Iteration : 364
Lower bound : 5.053479856251191
Valeur au point courant : 5.054474004438809
Iteration : 365
Lower bound : 5.053400036886522
Valeur au point courant : 5.0549984364763
Iteration : 366
Lower bound : 5.053439282327361
Valeur au point courant : 5.055089976848347
Iteration : 367
Lower bound : 5.05347428130621
Valeur au point courant : 5.054726933804579
Iteration : 368
Lower bound : 5.053493114881652
Valeur au point courant : 5.054978513823565
Iteration : 369
Lower bound : 5.053524096842498
Valeur au point courant : 5.0551088621174625
Iteration : 370
Lower bound : 5.053523328939486
Valeur au point courant : 5.055364539882465
Iteration : 371
Lower bound : 5.053461523922005
Valeur au point courant : 5.055201503069993
Iteration : 372
Lower bound : 5.053425683636321
Valeur au point courant : 5.055278156091857
Iteration : 373
Lower bound : 5.053473891241568
Valeur au point courant : 5.055439

...

Iteration : 1698
Lower bound : 5.05403547499935
Valeur au point courant : 5.054129052295303
Iteration : 1699
Lower bound : 5.054034649346997
Valeur au point courant : 5.054129187196015
Iteration : 1700
Lower bound : 5.054034936483456
Valeur au point courant : 5.054129266311467
Iteration : 1701
Lower bound : 5.054035171445655
Valeur au point courant : 5.054129082697018
Iteration : 1702
Lower bound : 5.054035348349595
Valeur au point courant : 5.054129282051837
Iteration : 1703
Lower bound : 5.054035733178467
Valeur au point courant : 5.054129788680884
Iteration : 1704
Lower bound : 5.0540368547054335
Valeur au point courant : 5.054130462409367
Iteration : 1705
Lower bound : 5.054037146213698
Valeur au point courant : 5.054131652389321
Iteration : 1706
Lower bound : 5.054036795151088
Valeur au point courant : 5.054130987276423
Iteration : 1707
Lower bound : 5.05403601133417
Valeur au point courant : 5.0541302067530705
Iteration : 1708
Lower bound : 5.054037034193652
Valeur au point coura

Iteration : 1786
Lower bound : 5.054045566685608
Valeur au point courant : 5.054131807788952
Iteration : 1787
Lower bound : 5.05404622078512
Valeur au point courant : 5.054133730345955
Iteration : 1788
Lower bound : 5.054046307091895
Valeur au point courant : 5.054132380767946
Iteration : 1789
Lower bound : 5.054046298962142
Valeur au point courant : 5.054131543294359
Iteration : 1790
Lower bound : 5.0540461903652805
Valeur au point courant : 5.054131015956089
Iteration : 1791
Lower bound : 5.054046884214065
Valeur au point courant : 5.054133334932541
Iteration : 1792
Lower bound : 5.054047158679493
Valeur au point courant : 5.054132119662491
Iteration : 1793
Lower bound : 5.0540468864567645
Valeur au point courant : 5.05413143846703
Iteration : 1794
Lower bound : 5.054047103456165
Valeur au point courant : 5.054133853560855
Iteration : 1795
Lower bound : 5.054047592138316
Valeur au point courant : 5.054131940695541
Iteration : 1796
Lower bound : 5.054047165920447
Valeur au point coura

Lower bound : 5.054055698928522
Valeur au point courant : 5.054133773026722
Iteration : 1875
Lower bound : 5.0540556788907836
Valeur au point courant : 5.054133441432026
Iteration : 1876
Lower bound : 5.054055804821939
Valeur au point courant : 5.05413359873949
Iteration : 1877
Lower bound : 5.054055467852038
Valeur au point courant : 5.0541333198627685
Iteration : 1878
Lower bound : 5.054055239616863
Valeur au point courant : 5.054133434941719
Iteration : 1879
Lower bound : 5.054055667244221
Valeur au point courant : 5.054134449500245
Iteration : 1880
Lower bound : 5.054056102853521
Valeur au point courant : 5.054134909325189
Iteration : 1881
Lower bound : 5.05405557524311
Valeur au point courant : 5.054135266665171
Iteration : 1882
Lower bound : 5.054055386133612
Valeur au point courant : 5.054135764301326
Iteration : 1883
Lower bound : 5.0540574672285405
Valeur au point courant : 5.05413571177486
Iteration : 1884
Lower bound : 5.054055716529455
Valeur au point courant : 5.0541355996

Iteration : 1963
Lower bound : 5.054004338971591
Valeur au point courant : 5.0541251913666825
Iteration : 1964
Lower bound : 5.0540029394851835
Valeur au point courant : 5.054123239294592
Iteration : 1965
Lower bound : 5.054001822904876
Valeur au point courant : 5.054121515671543
Iteration : 1966
Lower bound : 5.054001012213767
Valeur au point courant : 5.054120893021912
Iteration : 1967
Lower bound : 5.054008264900431
Valeur au point courant : 5.054122047388292
Iteration : 1968
Lower bound : 5.0540061763704855
Valeur au point courant : 5.054119693053549
Iteration : 1969
Lower bound : 5.054005410459121
Valeur au point courant : 5.05411760688338
Iteration : 1970
Lower bound : 5.054004572395119
Valeur au point courant : 5.054116023903221
Iteration : 1971
Lower bound : 5.054004148512835
Valeur au point courant : 5.054115154435038
Iteration : 1972
Lower bound : 5.054012527802202
Valeur au point courant : 5.054124990422496
Iteration : 1973
Lower bound : 5.054008434301959
Valeur au point cou

InterruptException: InterruptException:

## Bundle algorithm
Comparing to the cutting plane method, the bundle algorithm adds a
quadratic penalization to the polyhedral proxy model.
The function .
$$
\begin{aligned}
f_k(x) = min_x \;& \theta + \frac 12 \| x - x_k \|_2^2 \\
         s.t. \quad & \theta \geq g_k^\top (x - x_k) + f(x_k)
\end{aligned}
$$

In [41]:
function update_center(var, nVariables, center)
    JuMP.fix.(var, center)
end

function launch_bundle(X, xMin, xMax, maxit=10000)
    master = Model(with_optimizer(SOLVER))
    JuMP.set_silent(master)
    
    @variable(master, α >= LB)
    
    @variable(master, xMin[i] <= x[i in 1:size(X,1)] <= xMax[i])
    
    @variable(master, var[i in 1:size(X,1)]) #Proximal variable
    
    lb, ub = LB, UB

    best_ub = ub

    #trace_f = Float64[]
    #trace_ub = Float64[]
    #trace_α = Float64[]
    
    prediction = 0.0
    # Number of serious step
    nb_ss = 0
    # Number of null step
    nb_ns = 0
    # Maximum number of update
    nb_update = 3
    step = "NONE"
    weight = 1.0 #c = 0.5
    tol = 1e-6

    center = zeros(nVariables)
    update_center(var,nVariables,center)
    
    # The objective writes out as a QP
    @objective(master, Min, α + weight*sum((var[i]-x[i])^2 for i in 1:nVariables))
    
    for n_iter in 1:maxit
        JuMP.optimize!(master)
        println("Iteration ", n_iter)
        lb = JuMP.value(α)
        x_k = JuMP.value.(x)
        f_k = f(x_k)
        println("Lower bound : ", lb)
        println("Function value : ", f_k)
        if f_k - lb <= tol
            if norm(center-yi,2) <= tol
                println("Optimum reached within tolerance threshold.")
                return lb
            else
                #Serious step
                nb_ss += 1
                println("Serious step.")
                center = x_k
                update_center(var,nVariables,center)
            end    
        else
            if f_k - lb <= weight*norm(center-x_k,2)^2
                println("Optimum reached within tolerance threshold.")
                return lb
            else
                #Null step
                println("Null step.")
                nb_ns = nb_ns + 1
                g_k = ForwardDiff.gradient(f, x_k)
                @constraint(master, α >= f_k + sum(g_k[i]*(x-x_k)[i] for i in 1:size(X,1)))
            end
        end 
    end
    return lb
end

launch_bundle (generic function with 2 methods)

In [42]:
launch_bundle(A,xMin,xMax)

Iteration 1
Lower bound : -3.1800003999959984e9
Function value : 5.5995940672380655
Null step.
Iteration 2
Lower bound : -5958.029756755026
Function value : 4.435740864137699e6
Null step.
Iteration 3
Lower bound : -5140.694941529726
Function value : 2.155399487265759e6
Null step.
Iteration 4
Lower bound : -4367.306488792228
Function value : 1.5150248566501837e6
Null step.
Iteration 5
Lower bound : -3647.4963522306252
Function value : 760500.744284167
Null step.
Iteration 6
Lower bound : -3299.9002979576
Function value : 976153.476847548
Null step.
Iteration 7
Lower bound : -2948.645321984721
Function value : 475991.49179740687
Null step.
Iteration 8
Lower bound : -2649.9039672007575
Function value : 361112.08313188754
Null step.
Iteration 9
Lower bound : -2381.806449310492
Function value : 241408.637819267
Null step.
Iteration 10
Lower bound : -2023.3012889951933
Function value : 468942.23804678844
Null step.
Iteration 11
Lower bound : -1981.62474172191
Function value : 174252.82831591

Null step.
Iteration 94
Lower bound : -24.99785217971668
Function value : 143.71900295792165
Null step.
Iteration 95
Lower bound : -24.259870229428596
Function value : 94.08574790054544
Null step.
Iteration 96
Lower bound : -23.109319272954114
Function value : 137.36784537471806
Null step.
Iteration 97
Lower bound : -21.95270263195919
Function value : 134.02618121331193
Null step.
Iteration 98
Lower bound : -21.540677176009623
Function value : 103.58578687204266
Null step.
Iteration 99
Lower bound : -21.24800179955276
Function value : 87.11255480170645
Null step.
Iteration 100
Lower bound : -20.609430434515772
Function value : 64.25145250849624
Null step.
Iteration 101
Lower bound : -18.09269692120246
Function value : 104.63701143648643
Null step.
Iteration 102
Lower bound : -16.632285639052476
Function value : 98.4733511985134
Null step.
Iteration 103
Lower bound : -16.23569835733621
Function value : 101.49346429663746
Null step.
Iteration 104
Lower bound : -15.717257562492494
Functio

Iteration 187
Lower bound : 5.035074435795268
Function value : 5.102205713369774
Null step.
Iteration 188
Lower bound : 5.037896891184415
Function value : 5.084955873369792
Null step.
Iteration 189
Lower bound : 5.038852631335198
Function value : 5.096455067278689
Null step.
Iteration 190
Lower bound : 5.040722890331726
Function value : 5.11091309451228
Null step.
Iteration 191
Lower bound : 5.043330644188721
Function value : 5.106378746754159
Null step.
Iteration 192
Lower bound : 5.045128731186553
Function value : 5.076384012462604
Null step.
Iteration 193
Lower bound : 5.046247581443456
Function value : 5.066367061804251
Null step.
Iteration 194
Lower bound : 5.047604336681363
Function value : 5.069890835497201
Null step.
Iteration 195
Lower bound : 5.048381404873609
Function value : 5.066690030104894
Null step.
Iteration 196
Lower bound : 5.048846351309897
Function value : 5.084669011015849
Null step.
Iteration 197
Lower bound : 5.04956636526607
Function value : 5.070173929329853
N

5.054035669135729